# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("Berlin")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
from datetime import date, datetime

availabilities = {"london": "01/01/2025-31/03/2025", "paris": "01/01/2025-31/12/2025", "tokyo": "05/06/2024-04/06/2025", "berlin": "01/12/2024-30/09/2025"}

def get_availability_period(destination_city, flight_date):
    """Check for dates as LLM answers on this are not reliable."""
    print(f"Tool get_availability_period called for {destination_city}")
    
    city = destination_city.lower()
    availability = availabilities.get(city)
    
    if availability:
        current_date = date.today()
        if len(flight_date.split("-")) == 2:
            flight_date = f"{flight_date}-{str(current_date.year)}"
        flight_date = datetime.strptime(flight_date, "%d-%m-%Y").date()
        
        start_str, end_str = availability.split("-")
        start_date = datetime.strptime(start_str, "%d/%m/%Y").date()
        end_date = datetime.strptime(end_str, "%d/%m/%Y").date()
        
        if start_date < flight_date < end_date:
            return "Available"
    else:
        return "Not available for requested flight dates"

In [ ]:

availability_function = {
    "name": "get_availability_period",
    "description": "Check if the requested flight dates are within the availability period of the flight schedule to the destination city. Call this whenever you need to know whether there is a flight to the requested city at the provided travel dates of the user, for example when a customer asks 'I want to book a flight to Tokyo on 5 Nov 2025'. \
    If the user's date does not include a year, the date will refer to a date after the current date. State in your response whether there will be a flight to the destination city at the requested flight date. \
    For example, an availability of '01/08/2025-31/10/2025' means that there are flights between 01 Aug 25 and 31 Oct 2025. \
    Parse the user's flight date as string in the format DD-MM-YYYY or DD-MM if no year is given",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "flight_date": {
                "type": "string",
                "description": "The date on which the customer wants to travel",
            },
        },
        "required": ["destination_city", "flight_date"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": availability_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tools(message)
        messages.append(message)
        print(f"Message: \n {messages}")
        messages.append(response)
        print(f"Tool call response: \n {response}")
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
# Extended function to handle two tools

def handle_tools(message):
    print(f"message.tool_calls:\n {message.tool_calls}")
    tool_call = message.tool_calls[0]      
    if message.tool_calls[0].function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        content = json.dumps({"destination_city": city,"price": price})
    elif message.tool_calls[0].function.name == "get_availability_period":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        flight_date = arguments.get('flight_date')
        availability_dates = get_availability_period(city, flight_date)
        content = json.dumps({"destination_city": city,"availability_dates": availability_dates})
    
    response = {
            "role": "tool",
            "content": content,
            "tool_call_id": tool_call.id
    }

    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()